In [34]:
from __future__ import unicode_literals, print_function, division
from tensorboard_logger import configure, log_value
from code_S3R import my_nets, my_utils
import torch

hyper_params = {
    'net_type': 'xyz',
    'activation_fct': 'relu',
    'learning_rate': 1e-3,
    'num_epochs': 5,
    'batch_size': 32,
}


# Data
# -------------

# Load the dataset

x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = my_utils.load_data()

# Shuffle sequences and resize sequences
x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = my_utils.preprocess_data(x_train, x_test, y_train_14,
                                                                                y_train_28,
                                                                                y_test_14, y_test_28)

# Convert to pytorch variables
x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = my_utils.convert_to_pytorch_tensors(x_train, x_test,
                                                                                           y_train_14,
                                                                                           y_train_28, y_test_14,
                                                                                           y_test_28)


/Users/alexandre/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [48]:
print(x_train.size(), y_train_14.size())

torch.Size([1960, 100, 66]) torch.Size([1960])


In [36]:
# Prepare all mini-batches
x_train_batches = my_utils.batch(
    x_train,
    batch_size=hyper_params['batch_size']
)  # list of tensors (batch_size, temporal_duration, nb_sequences=66)
y_train_batches = my_utils.batch(
    y_test_14,
    batch_size=hyper_params['batch_size']
)  # list of output tensors containing the index of the right gesture, size : (batch_size)


In [52]:
print(x_train_batches[61].size(), len(x_train_batches), y_train_batches[0].size(), len(y_train_batches))

torch.Size([8, 100, 66]) 62 torch.Size([32]) 27


In [44]:
net = my_nets.XYZNet(activation_fct='relu')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=net.parameters(), lr=hyper_params['learning_rate'])


In [46]:
for i, (x_train_batch, y_train_batch) in enumerate(zip(x_train_batches, y_train_batches)):
    print(i, x_train_batch.size(), outputs.size(), y_train_batch.size())
    
    # zero the gradient parameters
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(
        x_train_batch)  # outputs is a tensor (batch_size, nb_classes=14) with the proba of each gesture
    loss = criterion(outputs, y_train_batch)
    loss.backward()
    optimizer.step()


0 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
1 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
2 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
3 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
4 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
5 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
6 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
7 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
8 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
9 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
10 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
11 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
12 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
13 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
14 torch.Size([32, 100, 66]) torch.Size([32, 14]) torch.Size([32])
15 to

ValueError: Expected input batch_size (32) to match target batch_size (5).